# Final Project
## Dude's Adventures
## McKenzee Masters CS-150

In [ ]:
import pygame
from pygame.locals import *
import os
import sys
import math
import random
pygame.init()

#Surface
W=600
H=400
surface = pygame.display.set_mode((W,H))
pygame.display.set_caption('Side Scroller')

#Background Imaging
bg = pygame.transform.scale(pygame.image.load('bg.png'), (W,H))
bgX = 0
bgX2 = bg.get_width()

clock = pygame.time.Clock()

#Character
class player(object):
    #Character Images
    run = [pygame.image.load(os.path.join('images', str(x) + '.png')) for x in range(8,16)]
    jump = [pygame.image.load(os.path.join('images', str(x) + '.png')) for x in range(1,8)]
    slide = [pygame.image.load(os.path.join('images', 'S1.png')),pygame.image.load(os.path.join('images', 'S2.png')),pygame.image.load(os.path.join('images', 'S2.png')),pygame.image.load(os.path.join('images', 'S2.png')), pygame.image.load(os.path.join('images', 'S2.png')),pygame.image.load(os.path.join('images', 'S2.png')), pygame.image.load(os.path.join('images', 'S2.png')), pygame.image.load(os.path.join('images', 'S2.png')), pygame.image.load(os.path.join('images', 'S3.png')), pygame.image.load(os.path.join('images', 'S4.png')), pygame.image.load(os.path.join('images', 'S5.png'))]
    fall = pygame.image.load(os.path.join('images','0.png'))
    jumpList = [1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,3,3,3,3,3,3,3,3,3,3,3,3,4,4,4,4,4,4,4,4,4,4,4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4]
    def __init__(self, x, y, width, height):
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        self.jumping = False
        self.sliding = False
        self.falling = False
        self.slideCount = 0
        self.jumpCount = 0
        self.runCount = 0
        self.slideUp = False
    
    #Character Movement 
    def draw(self, surface):
        if self.falling:
            surface.blit(self.fall, (self.x, self.y + 30))
                
        elif self.jumping:
            self.y -= self.jumpList[self.jumpCount] * 2
            surface.blit(self.jump[self.jumpCount//18], (self.x,self.y))
            self.jumpCount += 1
            if self.jumpCount > 108:
                self.jumpCount = 0
                self.jumping = False
                self.runCount = 0
            self.hitbox = (self.x+ 4,self.y,self.width-24,self.height-10)
        elif self.sliding or self.slideUp:
            if self.slideCount < 20:
                self.y += 1
                self.hitbox = (self.x+ 4,self.y,self.width-24,self.height-10)
            elif self.slideCount == 80:
                self.y -= 19
                self.sliding = False
                self.slideUp = True
            elif self.slideCount > 20 and self.slideCount < 80:
                self.hitbox = (self.x,self.y+3,self.width-8,self.height-35)
                
            if self.slideCount >= 110:
                self.slideCount = 0
                self.runCount = 0
                self.slideUp = False
                self.hitbox = (self.x+ 4,self.y,self.width-24,self.height-10)
            surface.blit(self.slide[self.slideCount//10], (self.x,self.y))
            self.slideCount += 1
            
        else:
            if self.runCount > 42:
                self.runCount = 0
            surface.blit(self.run[self.runCount//6], (self.x,self.y))
            self.runCount += 1
            self.hitbox = (self.x+ 4,self.y,self.width-24,self.height-13)
        #pygame.draw.rect(surface, (255,0,0),self.hitbox, 2)

#Obstacles
class up_spike(object):
    img1 = pygame.transform.scale(pygame.image.load(os.path.join('images', 'Spike1.png')), (60, 200))
    def __init__(self,x,y,width,height):
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        self.rotateCount = 0
        self.vel = 1.4

    def draw(self,surface):
        self.hitbox = (self.x+15, self.y , 30, self.height-10)
        #pygame.draw.rect(surface, (255,0,0), self.hitbox, 2)
        surface.blit(self.img1, (self.x,self.y))

    def collide(self, rect):
        #Checking if x-coordinates match
        if rect[0] + rect[2] > self.hitbox[0] and rect[0] < self.hitbox[0] + self.hitbox[2]:
            #Checking if y-coordinates match
            if rect[1] + rect[3] > self.hitbox[1]:
                return True
        else:
            return False


class down_spike(up_spike):
    img = pygame.transform.scale(pygame.image.load(os.path.join('images', 'Spike2.png')), (60,300))
    def draw(self,surface):
        self.hitbox = (self.x+5, self.y, 30, 300)
        #pygame.draw.rect(surface, (255,0,0), self.hitbox, 2)
        surface.blit(self.img, (self.x,self.y))

    def collide(self, rect):
        #Checking if x-coordinates match
        if rect[0] + rect[2] > self.hitbox[0] and rect[0] < self.hitbox[0] + self.hitbox[2]:
            #Checking if y-coordinates match
            if rect[1] < self.hitbox[3]:
                return True
        else:
            return False

#Text file to store highest Score
def updateFile():
    f = open('scores.txt','r')
    file = f.readlines()
    last = int(file[0])

    if last < int(score):
        f.close()
        file = open('scores.txt', 'w')
        file.write(str(score))
        file.close()

        return score
               
    return last



def endScreen():
    global pause, score, speed, obstacles
    pause = 0
    speed = 60
    obstacles = []
                   
    run = True
    while run:
        pygame.time.delay(100)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
                pygame.quit()
            if event.type == pygame.MOUSEBUTTONDOWN:
                run = False
                runner.falling = False
                runner.sliding = False
                runner.jumpin = False
                
        surface.blit(bg, (0,0))
        largeFont = pygame.font.SysFont('comicsans', 80)
        lastScore = largeFont.render('Best Score: ' + str(updateFile()),1,(255,255,255))
        currentScore = largeFont.render('Score: '+ str(score),1,(255,255,255))
        surface.blit(lastScore, (W/2 - lastScore.get_width()/2,150))
        surface.blit(currentScore, (W/2 - currentScore.get_width()/2, 240))
        pygame.display.update()
    score = 0

        


def redrawWindow():
    surface.blit(bg, (bgX, 0))
    surface.blit(bg, (bgX2,0))
    runner.draw(surface)
    for obstacle in obstacles:
        obstacle.draw(surface)

    pygame.display.update()

#Timers 
pygame.time.set_timer(USEREVENT+1, 500) #Background timer
pygame.time.set_timer(USEREVENT+2, 3000) #Obstacle timer
speed = 80

score = 0

active = True
runner = player(175, 280, 64, 64)

obstacles = []
pause = 0
fallSpeed = 0

while active:
    if pause > 0:
        pause += 1
        if pause > fallSpeed * 2:
            endScreen()
        
    score = speed//10 - 3

    for obstacle in obstacles:
        if obstacle.collide(runner.hitbox):
            runner.falling=True
            
            
            if pause == 0:
                pause = 1
                fallSpeed = speed
        if obstacle.x < -64:
            obstacles.pop(obstacles.index(obstacle))
        else:
            obstacle.x -= 1.4
    
    #Moving Background
    bgX -= 1.4
    bgX2 -= 1.4
    if bgX < bg.get_width() * -1:
        bgX = bg.get_width()
        #resets position of background 
    if bgX2 < bg.get_width() * -1:
        bgX2 = bg.get_width() 
        #resets position of background
        
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            active = False
            
        if event.type == USEREVENT+1:
            speed += 1 #increasing bg speed
            
        if event.type == USEREVENT+2: #timer for obstacles
            r = random.randrange(0,2)
            #storing obstacles in a list
            if r == 0:
                obstacles.append(up_spike(610, 200, 60, 120))
            elif r == 1:
                obstacles.append(down_spike(610, 0, 60, 120))
                
    if runner.falling == False:
        keys = pygame.key.get_pressed()
        #Jumping
        if keys[pygame.K_SPACE] or keys[pygame.K_UP]:
            if not(runner.jumping):
                runner.jumping = True
        #Sliding
        if keys[pygame.K_DOWN]:
            if not(runner.sliding):
                runner.sliding = True

    clock.tick(speed)
    redrawWindow()